In [ ]:
# ! pip install xclim 

In [ ]:
%matplotlib inline
import xarray as xr
import numpy as np 
import os 
import pandas as pd 
import matplotlib.pyplot as plt 

import intake
import zarr

import gcsfs

from xclim.core.calendar import convert_calendar

In [65]:
import warnings
warnings.filterwarnings("ignore")

For bias correction, we will be using 1995-2014 (the CMIP6 reference period). 

In [2]:
years = np.arange(1995, 2015)

function for removing leap days and converting the calendar to `noleap`

In [67]:
def convert_to_noleap_calendar(ds, target='noleap'):
    ds_noleap = convert_calendar(ds, target=target)
    return ds_noleap 

In [5]:
import dask
import dask.array as da
import dask.distributed as dd
import rhg_compute_tools.kubernetes as rhgk

In [ ]:
fs = gcsfs.GCSFileSystem(token='/opt/gcsfuse_tokens/impactlab-data.json')

In [6]:
client, cluster = rhgk.get_standard_cluster()

In [7]:
cluster

In [68]:
era5_variables = ['tmax', 'tmin', 'precip_total']

In [70]:
def create_era5_zarr_store(variable, years): 
    
    # create list of filenames to load
    filenames = [os.path.join('/gcs/impactlab-data/climate/source_data/ERA-5/day/%s/v1.1' %variable, 
                          '%s_daily_%s-%s.nc' %(variable, year, year)) for year in years]
    
    # Load files for specified years, note that the `preprocess=convert_to_noleap_calendar` argument is 
    # removing leap days and updating the calendar on each yearly file of daily data as it's being concatenated
    ds_allyears = xr.open_mfdataset(filenames, 
                                preprocess=convert_to_noleap_calendar)
    store_filename = 'gs://impactlab-data/climate/source_data/ERA-5/downscaling/%s.1995-2014.0p25.zarr' %variable 
    store = fs.get_mapper(store_filename, check=False)
    
    if not fs.exists(store_filename): 
        # save as a zarr store for rechunking 
        ds_allyears.to_zarr(store, consolidated=True, mode="w")

create zarr stores for tmax, tmin and precip_total

In [71]:
for variable in era5_variables: 
    create_era5_zarr_store(variable, years)

check to be sure xarray can read one of the test zarr stores

In [60]:
store_filename = 'gs://impactlab-data/climate/source_data/ERA-5/zarr/%s.1995-2014.0p25_test.zarr' %variable 
store = fs.get_mapper(store_filename, check=False)
ds_testread = xr.open_zarr(store)

In [61]:
ds_testread

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 7300)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) object 1995-01-01 00:00:00 ... 2014-12-31 00:00:00
Data variables:
    tmax       (time, latitude, longitude) float32 dask.array<chunksize=(365, 721, 1440), meta=np.ndarray>
Attributes:
    author:   Meredith Fish
    contact:  meredith.fish@rutgers.edu
    created:  2021-02-16
    project:  impactlab-rhg/climate/source-data/ERA-5
    source:   impactlab-rhg/climate/downscaled/ERA-5/hourly